## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-04-10-53-28 +0000,bbc,Investigation into pre-Budget leaks is under w...,https://www.bbc.com/news/articles/c4g9e7wp79zo...
1,2025-12-04-10-51-11 +0000,nyt,Ruined Rice Fields and Broken Rail Lines: Sri ...,https://www.nytimes.com/2025/12/04/world/asia/...
2,2025-12-04-10-50-35 +0000,bbc,Streeting orders review into mental health and...,https://www.bbc.com/news/articles/ce8q26q2r75o...
3,2025-12-04-10-49-37 +0000,bbc,Ukraine and US negotiators to meet in Florida ...,https://www.bbc.com/news/articles/c7731n5z6d3o...
4,2025-12-04-10-49-00 +0000,wsj,Eurozone Retail Sales Unchanged in October,https://www.wsj.com/world/europe/eurozone-reta...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2356/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
152,trump,56
42,new,23
25,ukraine,15
77,putin,14
544,he,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
219,2025-12-03-22-05-00 +0000,wsj,"Rep. Elise Stefanik, a top Trump ally, critici...",https://www.wsj.com/politics/elise-stefanik-mi...,113
39,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...,109
146,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...,108
144,2025-12-04-00-11-47 +0000,nyt,Trump Administration Begins Immigration Operat...,https://www.nytimes.com/2025/12/03/us/border-p...,96
361,2025-12-03-14-11-30 +0000,nypost,‘Grim Reaper’ White House budget chief Russ Vo...,https://nypost.com/2025/12/03/us-news/wh-budge...,94


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
219,113,2025-12-03-22-05-00 +0000,wsj,"Rep. Elise Stefanik, a top Trump ally, critici...",https://www.wsj.com/politics/elise-stefanik-mi...
151,82,2025-12-04-00-00-00 +0000,wsj,Defense Secretary Pete Hegseth shocked officia...,https://www.wsj.com/politics/national-security...
6,69,2025-12-04-10-39-00 +0000,wsj,"New York Times Sues Hegseth, Defense Departmen...",https://www.wsj.com/business/media/new-york-ti...
369,63,2025-12-03-13-33-53 +0000,nypost,Arrogant Putin sees ‘no point in making any se...,https://nypost.com/2025/12/03/world-news/putin...
118,51,2025-12-04-02-17-53 +0000,nypost,"Family of boy, 11, allegedly killed by illegal...",https://nypost.com/2025/12/03/us-news/family-o...
122,47,2025-12-04-01-55-16 +0000,nypost,Halle Berry bashes Gov. Gavin Newsom at NYT su...,https://nypost.com/2025/12/03/us-news/halle-be...
39,43,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...
388,40,2025-12-03-11-39-49 +0000,nypost,Macron arrives in China for talks with Xi on t...,https://nypost.com/2025/12/03/world-news/macro...
274,38,2025-12-03-19-37-05 +0000,latimes,Former sheriff's deputy killed after police ch...,https://www.latimes.com/california/story/2025-...
300,36,2025-12-03-18-05-36 +0000,nypost,Private employers cut 32K jobs last month — hi...,https://nypost.com/2025/12/03/business/private...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
